In [4]:
import numpy as np
import pickle
import os
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
from scipy import signal


In [5]:
def re_im_sep(fields):
    shg1 = fields[0:1892] + fields[1892 * 2 + 348:1892 * 3 + 348] * 1j
    shg2 = fields[1892:1892 * 2] + fields[1892 * 3 + 348:1892 * 4 + 348] * 1j
    sfg = fields[1892 * 2:1892 * 2 + 348] + fields[1892 * 4 + 348:1892 * 4 + 2 * 348] * 1j

    return shg1, shg2, sfg

def change_domains(domain, field, new_domain, domain_type):
    padded_vector = np.pad(field, (1, 1), mode='constant')
    window = signal.windows.tukey(int(len(padded_vector) // 1))
    padded_vector = window * padded_vector

    # Extend old domain to match new domain
    alt_domain = np.append([new_domain[0]], domain)
    alt_domain = np.append(alt_domain, [new_domain[-1]])

    # Resample padded vector using new domain
    resampled_vector = resample_method1(alt_domain, new_domain, padded_vector)
    

    if (domain_type == "freq"):
        out_direct = ifft(field)
        out = ifft(resampled_vector)
    elif (domain_type == "time"):
        out_direct = fft(field)
        out = fft(resampled_vector)
    else:
        print("field type not supported")

    return out_direct, out


def change_domain_and_adjust_energy(domain, field, new_domain, domain_type, beam_area, domain_spacing,
                                    true_domain_spacing):
    out_direct, out = change_domains(domain, field, new_domain, domain_type)
    domain_spacing_calc = domain[1] - domain[0]
    pulse_energy_calc = calc_energy_expanded(field, domain_spacing_calc, beam_area)
    out_direct = energy_match_expanded(out_direct, pulse_energy_calc, domain_spacing, beam_area)
    out = energy_match_expanded(out, pulse_energy_calc, true_domain_spacing, beam_area)

    print("direct, and resampled energies: ", calc_energy_expanded(out_direct, domain_spacing, beam_area),
          calc_energy_expanded(out, true_domain_spacing, beam_area))

    return out_direct, out


def change_domain_timeFreq(t):
    # Calculate time step and sample frequency
    dt = t[1] - t[0]
    fs = 1 / dt

    # Calculate frequency vector using fftfreq
    n = len(t)
    freqs = np.fft.fftfreq(n, dt)

    # Shift frequency vector so that 0 Hz is in the center
    freqs = np.fft.fftshift(freqs)

    return freqs

def intensity_phase_plot(domains, fields, labels, colors, domain_type, xlims=None, ylabel=None, normalize=False, legend=False,offsets=None,
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False):
    """
    Plot intensity and phase of a field
    """
    if domain_type == "time":
            factor = 1e12
            xlabel = "time (ps)"
    elif domain_type == "wavelength":
        factor = 1e9
        xlabel = "wavelength (nm)"
    elif domain_type == "frequency" or domain_type == "freq":
        factor = 1e-12
        xlabel = "frequency (THz)"
    for ii in range(len(domains)):
        domains[ii] = domains[ii]*factor

    intensities = [get_intensity(field) for field in fields]
    phases = [np.unwrap(get_phase(field)) for field in fields]

    

    ylabel2 = "Phase (rad)"
    #TODO: get clear working so that can call function multiple times and have all plots appear
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 4),clear=False)

    axs.set_xlabel(xlabel)

    
    
    for ii in range(len(intensities)):
        if normalize:
            intensity = intensities[ii] / np.max(intensities[ii])
            ylabel1 = "Norm. Intensity (a.u.)"
        else:
            ylabel1 = "Fluence (J/m^2)"
#             warnings.warn("Using default intensity units (J/m^2)")
            intensity = intensities[ii]

        if ylabel is None:
            ylabel1 = ylabel1
        else:
            ylabel1 = ylabel
        if offsets is not None:
            offset = offsets[ii]
        axs.plot(domains[ii], intensity+offset, color=colors[ii], label=labels[ii],alpha=.6)
    if (legend):
        plt.legend()
    if xlims is not None:
        axs.set_xlim(xlims[0], xlims[1])
    axs.set_ylabel(ylabel1, color="black")
    ax2 = axs.twinx()
    for ii in range(len(intensities)):
        
        ax2.plot(domains[ii], phases[ii], color=colors[ii], linestyle="dashed",alpha=.6)

    ax2.set_ylabel(ylabel2, color="black")

    if save:
        if save_name is None:
            raise ValueError("Save name is not specified")
        if "." + save_format not in save_name:
            save_name += "." + save_format
        plt.savefig(save_name, bbox_inches='tight', dpi=300, transparent=True, format=save_format)
    if plot_show:
        plt.show()
    else:
        if ~plot_hold:
            plt.close()

In [3]:
saved_model_output_dir = "" #directory from model training
data_directory = "" #directory from preprocessing

train_losses = np.load(os.path.join(saved_model_output_dir, "train_losses.npy"))
val_losses = np.load(os.path.join(saved_model_output_dir, "val_losses.npy"))

all_preds = np.load(os.path.join(saved_model_output_dir, "all_preds.npy")
all_preds_trans = np.load(os.path.join(saved_model_output_dir, "all_preds_transformed.npy")

freq_vectors_shg1 = np.load(
    "../Data/shg_freq_domain_ds.npy")
freq_vectors_shg2 = freq_vectors_shg1 # these are equivalent here
freq_vectors_sfg = np.load(
    "../Data/sfg_freq_domain_ds.npy")

domain_spacing_1 = (freq_vectors_shg1[1] - freq_vectors_shg1[0]) * 1e12 #scaled to be back in Hz
domain_spacing_2 = (freq_vectors_shg2[1] - freq_vectors_shg2[0]) * 1e12
domain_spacing_3 = (freq_vectors_sfg[1] - freq_vectors_sfg[0]) * 1e12

factors_freq = {"beam_area": 400e-6 ** 2 * np.pi,
                "grid_spacing": [domain_spacing_1, domain_spacing_2, domain_spacing_3],
                "domain_spacing_1": domain_spacing_1, "domain_spacing_2": domain_spacing_2,
                "domain_spacing_3": domain_spacing_3} #beam radius 400 um (and circular beam)
                          
                          
y_90 = np.load(os.path.join(data_directory, "y_new_90.npy") #these are used to compare to the predictions


with open(os.path.join(data_directory, 'scaler.pkl'), 'rb') as file: #can use scaler.pkl or scaler_bckkup.pkl
    scaler = pickle.load(file)
               
y_90_trans  = scaler.inverse_transform(y_90)

sfg_original_freq= np.load("../sfg_original_freq_vector.npy")
sfg_original_time= np.load("../sfg_original_time_vector.npy")
sfg_original_time_ds = sfg_original_time[1]-sfg_original_time[0]


FileNotFoundError: [Errno 2] No such file or directory: 'train_losses.npy'

In [ ]:
ii = 8 #use this to select one of the examples (should be 100 in total to choose among)

y_pred_trans = all_preds_trans[ii]
y_true_trans = y_90_trans[99:][::100][ii]

y_pred_trans_shg1, y_pred_trans_shg2, y_pred_trans_sfg = re_im_sep(y_pred_trans)
y_true_trans_shg1, y_true_trans_shg2, y_true_trans_sfg = re_im_sep(y_true_trans)


sfg_freq_to_time_direct_pred, sfg_freq_to_time_pred=change_domain_and_adjust_energy(freq_vectors_sfg, y_pred_trans_sfg, sfg_original_freq, "freq", beam_area=factors_freq["beam_area"],domain_spacing=domain_spacing_3, true_domain_spacing=sfg_original_time_ds)
sfg_freq_to_time_direct_true, sfg_freq_to_time_true=change_domain_and_adjust_energy(freq_vectors_sfg, y_true_trans_sfg, sfg_original_freq, "freq", beam_area=factors_freq["beam_area"],domain_spacing=domain_spacing_3, true_domain_spacing=sfg_original_time_ds)


shg1_freq_to_time_direct_pred, shg1_freq_to_time_pred=change_domain_and_adjust_energy(freq_vectors_shg1, y_pred_trans_shg1, sfg_original_freq, "freq", beam_area=factors_freq["beam_area"],domain_spacing=domain_spacing_1, true_domain_spacing=sfg_original_time_ds)
shg1_freq_to_time_direct_true, shg1_freq_to_time_true=change_domain_and_adjust_energy(freq_vectors_shg1, y_true_trans_shg1, sfg_original_freq, "freq", beam_area=factors_freq["beam_area"],domain_spacing=domain_spacing_1, true_domain_spacing=sfg_original_time_ds)

shg2_freq_to_time_direct_pred, shg2_freq_to_time_pred=change_domain_and_adjust_energy(freq_vectors_shg2, y_pred_trans_shg2, sfg_original_freq, "freq", beam_area=factors_freq["beam_area"],domain_spacing=domain_spacing_2, true_domain_spacing=sfg_original_time_ds)
shg2_freq_to_time_direct_true, shg2_freq_to_time_true=change_domain_and_adjust_energy(freq_vectors_shg2, y_true_trans_shg2, sfg_original_freq, "freq", beam_area=factors_freq["beam_area"],domain_spacing=domain_spacing_2, true_domain_spacing=sfg_original_time_ds)




In [ ]:
#training and validation error
plt.scatter(range(1, train_losses.shape[0]+1, 1), train_losses, label="Train Loss", alpha=0.6)
plt.scatter(range(1, val_losses.shape[0]+1, 1), val_losses, label="Validation Loss", alpha=0.6)
plt.title("MSE Loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()

In [ ]:
#plots frequency domain for all three fields (prediction vs true) normalized (first three) and non-normalized (next three)
print("------- Normalized True vs Prediction Frequency Domain --------")
print("*** SFG ***")

intensity_phase_plot([freq_vectors_sfg,freq_vectors_sfg], [y_test_trans_sfg,y_pred_trans_sfg], ["true","pred"], ["red","black"], "freq", normalize=True, legend=True,offsets=[0,.2],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)

print("*** SHG1 ***")
intensity_phase_plot([freq_vectors_shg1,freq_vectors_shg1], [y_test_trans_shg1,y_pred_trans_shg1], ["true","pred"], ["red","black"], "freq", normalize=True, legend=False,offsets=[0,.2],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)

print("*** SHG2 ***")
intensity_phase_plot([freq_vectors_shg2,freq_vectors_shg2], [y_test_trans_shg2,y_pred_trans_shg2], ["true","pred"], ["red","black"], "freq", normalize=True, legend=False,offsets=[0,.2],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)

print("------- Non-normalized True vs Prediction Frequency Domain --------")
print("*** SFG ***")
intensity_phase_plot([freq_vectors_sfg,freq_vectors_sfg], [y_test_trans_sfg,y_pred_trans_sfg], ["true","pred"], ["red","black"], "freq", normalize=False, legend=True,offsets=[0,0],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)
print("*** SHG1 ***")
intensity_phase_plot([freq_vectors_shg1,freq_vectors_shg1], [y_test_trans_shg1,y_pred_trans_shg1], ["true","pred"], ["red","black"], "freq", normalize=False, legend=False,offsets=[0,0],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)
print("*** SHG2 ***")
intensity_phase_plot([freq_vectors_shg2,freq_vectors_shg2], [y_test_trans_shg2,y_pred_trans_shg2], ["true","pred"], ["red","black"], "freq", normalize=False, legend=False,offsets=[0,0],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)


#plots time domain for all three fields (prediction vs true) normalized (first three) and non-normalized (next three)
print("------- Normalized True vs Prediction Time Domain --------")
print("*** SFG ***")
intensity_phase_plot([sfg_original_time,sfg_original_time], [sfg_freq_to_time_true,sfg_freq_to_time_pred], ["true","pred"], ["red","black"], "time", xlims=[-15,15], normalize=True, legend=True,offsets=[0,.2],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)
print("*** SHG1 ***")
intensity_phase_plot([sfg_original_time,sfg_original_time], [shg1_freq_to_time_true,shg1_freq_to_time_pred], ["true","pred"], ["red","black"], "time", normalize=True, legend=False,offsets=[0,.2],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)
print("*** SHG2 ***")
intensity_phase_plot([sfg_original_time,sfg_original_time], [shg2_freq_to_time_true,shg2_freq_to_time_pred], ["true","pred"], ["red","black"], "time", normalize=True, legend=False,offsets=[0,.2],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)

print("------- Non-normalized True vs Prediction Frequency Domain --------")
print("*** SFG ***")
intensity_phase_plot([sfg_original_time,sfg_original_time], [sfg_freq_to_time_true,sfg_freq_to_time_pred], ["true","pred"], ["red","black"], "time", xlims=[-15,15], normalize=False, legend=True,offsets=[0,0],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)
print("*** SHG1 ***")
intensity_phase_plot([sfg_original_time,sfg_original_time], [shg1_freq_to_time_true,shg1_freq_to_time_pred], ["true","pred"], ["red","black"], "time", normalize=False, legend=False,offsets=[0,0],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)
print("*** SHG2 ***")
intensity_phase_plot([sfg_original_time,sfg_original_time], [shg2_freq_to_time_true,shg2_freq_to_time_pred], ["true","pred"], ["red","black"], "time", normalize=False, legend=False,offsets=[0,0],
                                save_format="pdf", save_name=None, save=False, plot_show=True,plot_hold=False)



